In [73]:
# imports
%load_ext autoreload
%autoreload 2
import torch.nn as nn
import torch
import sys
import numpy as np
from tqdm import tqdm 
import cv2
from PIL import Image
import torchvision
from torchvision import transforms as pth_transforms
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from RevenuePredictorViT import RevenuePredictorViT
sys.path.append('dino')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
args_vit = {"arch": "vit_small",
 "patch_size": 8,
 "out_dim": 65536,
 "norm_last_layer": False,
 "warmup_teacher_temp": 0.04,
 "teacher_temp": 0.07,
 "warmup_teacher_temp_epochs": 30,
 "use_fp16": False,
 "weight_decay": 0.04,
 "weight_decay_end": 0.4,
 "clip_grad": 0,
 "batch_size_per_gpu": 64,
 "epochs": 800,
 "freeze_last_layer": 1,
 "lr": 0.0005,
 "warmup_epochs": 10,
 "min_lr": 1e-05,
 "global_crops_scale": [0.25,
 1.0],
 "local_crops_scale": [0.05,
 0.25],
 "local_crops_number": 10,
 "seed": 0,
 "num_workers": 10,
 "world_size": 16,
 "ngpus": 8,
 "nodes": 2,
 "optimizer": "adamw",
 "momentum_teacher": 0.996,
 "use_bn_in_head": False,
 "drop_path_rate": 0.1,
 "image_path": "./test_img/airbud.jpeg",
 "threshold": None,
 "image_size": (480, 480),
 "output_dir": "./"
}

args = {
   "avgpool": False,
    "last_n_blocks": 4, # TODO change this, this is what eval linear does
    "n_dnn_img_features": 16 
}

# Load Data

In [ ]:
# read csv (first 10 entries)


# Test one input

In [68]:
rp_ViT = RevenuePredictorViT(args, args_vit)

Pretrained weights found and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.projection_head.0.weight', 'head.projection_head.0.bias', 'head.projection_head.2.weight', 'head.projection_head.2.bias', 'head.projection_head.4.weight', 'head.projection_head.4.bias', 'head.prototypes.weight'])
Pretrained weights found and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.projection_head.0.weight', 'head.projection_head.0.bias', 'head.projection_head.2.weight', 'head.projection_head.2.bias', 'head.projection_head.4.weight', 'head.projection_head.4.bias', 'head.prototypes.weight'])


In [69]:
# set to GPU if possible
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# load image
image_path = "./test_img/airbud.jpeg"
with open(image_path, 'rb') as f:
    img = Image.open(f)
    img = img.convert('RGB')
transform = pth_transforms.Compose([
    pth_transforms.Resize(args_vit['image_size']),
    pth_transforms.ToTensor(),
    pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
img = transform(img)

# make the image divisible by the patch size
w, h = img.shape[1] - img.shape[1] % args_vit['patch_size'], img.shape[2] - img.shape[2] % args_vit['patch_size']
img = img[:, :w, :h].unsqueeze(0)

w_featmap = img.shape[-2] // args_vit['patch_size']
h_featmap = img.shape[-1] // args_vit['patch_size']

#rp_ViT.ViT_teacher(img.to(device)).shape
#intermediate_output = rp_ViT.ViT_teacher.get_intermediate_layers(img.to(device), 4)

In [72]:
features = torch.tensor([1,2,3])
rp_ViT(img, features)

#rp_ViT.vit_teacher(imgs)
# TODO figure out how to use the dataloader for multiple images

torch.Size([1, 16])

# Training Loop